### The Tic Tac Toe Pad and Dictionary

In [1]:
# This is the initial Pad
defaultPad = [['Lt','t ','Rt'],['L ','m ','R '],['Lb','b ','Rb']]

# Tic Tac Toe dictionary, keys are strings, values are matrix indexes (or point coordinates)
ticTacDict = {'LT':(0,0), 'T':(0,1), 'RT':(0,2),\
              'L':(1,0), 'M':(1,1), 'R':(1,2),\
              'LB':(2,0), 'B':(2,1), 'RB':(2,2)}

###### Printing

In [2]:
# pad printing function (uses global variables: defaultPad, cpu and player_choices)
def printPad(cpuHistory,playerHistory,pad=defaultPad):
    nRows = len(pad)
    
    # check for the player options
    if playerHistory:
        for i in range(len(playerHistory)):
            pad[playerHistory[i][0]][playerHistory[i][1]] = 'O '
            
    # check for CPU options
    if cpuHistory:
        for i in range(len(cpuHistory)):
            pad[cpuHistory[i][0]][cpuHistory[i][1]] = 'X '
    
    # print Pad!
    for i in range(3):
        print(pad[i])

###### Input: who's beginning the game?

In [3]:
# Input who's begining the game
def input_first_choice():
    
    firstChoice = input("Who will begin the game: Cpu or You? (Type: cpu or me, case not-sensitive) ")

    while True:
        # check input
        if firstChoice.lower().strip(" ") in ['cpu','me']:
            return firstChoice.lower().strip(" ")
        else:
            firstChoice = input("Please type cpu or me (case not-sensitive)!")

###### Input player

In [4]:
# input player function
def input_player_choice(cpuHistory,playerHistory,gameDict=ticTacDict):
    
    print('This is the game now:')
    printPad(cpuHistory,playerHistory)
    yourChoice = input("Choose your mark! (e.g: 'L' stands for 'Left') ")

    while True:
        # check if input is among dictionary keys
        if yourChoice.upper().strip(" ") in gameDict.keys():
            # check if input has already been selected
            if gameDict[yourChoice.upper().strip(" ")] in (playerHistory + cpuHistory):
                yourChoice = input("Choose another one! It has been already chosen! ")
            else:
                return gameDict[yourChoice.upper().strip(" ")]
        else:
            yourChoice = input("Select a proper position! ")

###### Cpu: random move!

In [5]:
from random import randint

# function generating a valid random choice for the cpu move
def gen_random_move(cpuHistory,playerHistory):
    while True:
        pos0 = randint(0,2)
        pos1 = randint(0,2)
        if len(playerHistory+cpuHistory) == 9:
            # break to avoid infinite loop in case I get here
            break
        elif (pos0,pos1) not in (playerHistory + cpuHistory):
            return (pos0,pos1)

###### Cpu: predict next move!
<img src="./images/ticTac.png" title="Determine direction"/>

In [6]:
# function predicting possible player/cpu win !
# it returns all the predicted wining pair of tuples (could be further optimized?)
def predict_wining(movesHistory):
    tuples2win = []
    nMoves = len(movesHistory)
    for i in range(nMoves):
        for j in range(nMoves):
            if j > i:
                # check horizontal or vertical
                if (movesHistory[i][0] == movesHistory[j][0]) | \
                (movesHistory[i][1] == movesHistory[j][1]):
                    tuples2win.append((movesHistory[i],movesHistory[j]))
                # check one Tic Tac Toe possible diagonal
                elif (movesHistory[i][0] == movesHistory[i][1]) & \
                (movesHistory[j][0] == movesHistory[j][1]):
                    tuples2win.append((movesHistory[i],movesHistory[j]))
                # check the other Tic Tac Toe possible diagonal
                elif (sum(movesHistory[i]) == 2) & (sum(movesHistory[j]) == 2):
                    tuples2win.append((movesHistory[i],movesHistory[j]))
    return tuples2win

###### Why is the direction between two points in the Tic Tac Toe Pad?
<img src="./images/direction.png" title="Determine direction"/>

In [7]:
# determine the direction between two-points in 2D
def determine_direction(pt1,pt0):
    x1_0 = pt1[0] - pt0[0]
    y1_0 = pt1[1] - pt0[1]
    segLen = (x1_0**2 + y1_0**2)**0.5
    cosTheta = x1_0/segLen
    sinTheta = y1_0/segLen
    return (cosTheta,sinTheta)

###### Which is the point that will cancel next wining move?

In [8]:
from numpy import negative

# function asigning cpu choice either to cancel player next wining move
def getWiningChoice(tuples2win,gameDict=ticTacDict):
    (cosTheta,sinTheta) = determine_direction(*tuples2win)
    
    for aTuple in gameDict.values():
        if (aTuple != tuples2win[0]) & (aTuple != tuples2win[1]):
            (cosGamma,sinGamma) = determine_direction(tuples2win[1],aTuple)
            
            if ((cosTheta,sinTheta) == (cosGamma,sinGamma)) | \
            ((cosTheta,sinTheta) == negative((cosGamma,sinGamma))).all():
                return aTuple

###### Is there a winner?

In [9]:
# this is the winner function
def determine_winner(movesHistory,gameDict=ticTacDict):
    nMoves = len(movesHistory)
    if nMoves >= 3:
        for i in range(nMoves):
            for j in range(nMoves):
                if j > i:
                    (cosTheta,sinTheta) = determine_direction(movesHistory[j],movesHistory[i])
                for k in range(nMoves):
                    if (j > i) & (k > j):
                        (cosGamma,sinGamma) = determine_direction(movesHistory[k],movesHistory[j])
                        if ((cosTheta,sinTheta) == (cosGamma,sinGamma)) | \
                        ((cosTheta,sinTheta) == negative((cosGamma,sinGamma))).all():
                            return True
    return False

###### Initialization of variables for the game!

In [10]:
# number of marked spaces in the pad
markedSpaces = 0

# Who will begin?
firstChoice = input_first_choice()

# Initialize cpu history of choices
if firstChoice == 'cpu':
    # First cpu choice default --> center of the pad
    cpu_choices = [(1,1)]
    markedSpaces += 1
else:
    # Initialize empty
    cpu_choices = []

# Initialize the player history of choices
player_choices = []

Who will begin the game: Cpu or You? (Type: cpu or me, case not-sensitive) me


###### This is the program loop!

In [11]:
# While loop is broken inside
while True:
    
    playerChoice = input_player_choice(cpu_choices,player_choices)
    player_choices.append(playerChoice)
    markedSpaces += 1
    # check player winner to break
    playerWins = determine_winner(player_choices)
    if playerWins:
        break
    elif markedSpaces == 9:
        # all marked Spaces --> break
        break
    
    # cpu looking for its wining chance
    tuples2win = predict_wining(cpu_choices)
    if tuples2win:
        # pick always the last pair of tuples
        cpu_choice = getWiningChoice(tuples2win[-1])
    else:
        # cpu just does a random move
        cpu_choice = gen_random_move(cpu_choices,player_choices)
    
    if (not cpu_choices) & ((1,1) not in player_choices):
        # Cpu first choice always check the center
        cpu_choice = (1,1)
    elif cpu_choice in player_choices:
        # Over-write cpu_choice in case is already chosen by player
        cpu_choice = gen_random_move(cpu_choices,player_choices)
    
    cpu_choices.append(cpu_choice)
    markedSpaces += 1
    # check cpu winner to break
    cpuWins = determine_winner(cpu_choices)
    if cpuWins:
        break
    elif markedSpaces == 9:
        # all marked Spaces --> break
        break

This is the game now:
['Lt', 't ', 'Rt']
['L ', 'm ', 'R ']
['Lb', 'b ', 'Rb']
Choose your mark! (e.g: 'L' stands for 'Left') m
This is the game now:
['Lt', 't ', 'Rt']
['L ', 'O ', 'R ']
['X ', 'b ', 'Rb']
Choose your mark! (e.g: 'L' stands for 'Left') lt
This is the game now:
['O ', 't ', 'X ']
['L ', 'O ', 'R ']
['X ', 'b ', 'Rb']
Choose your mark! (e.g: 'L' stands for 'Left') l
This is the game now:
['O ', 'X ', 'X ']
['O ', 'O ', 'R ']
['X ', 'b ', 'Rb']
Choose your mark! (e.g: 'L' stands for 'Left') b
This is the game now:
['O ', 'X ', 'X ']
['O ', 'O ', 'R ']
['X ', 'O ', 'X ']
Choose your mark! (e.g: 'L' stands for 'Left') r


###### Game conclusion!!!

In [12]:
if cpuWins:
    print("You have been beaten :(")
    printPad(cpu_choices,player_choices)
elif playerWins:
    print("You are the champion, my friend :)")
    printPad(cpu_choices,player_choices)
else:
    print("Tic Tac Toe Tied!!! Play again :)")
    printPad(cpu_choices,player_choices)

You are the champion, my friend :)
['O ', 'X ', 'X ']
['O ', 'O ', 'O ']
['X ', 'O ', 'X ']
